# CS6140 Assignments

**Instructions**
1. In each assignment cell, look for the block:
 ```
  #BEGIN YOUR CODE
  raise NotImplementedError.new()
  #END YOUR CODE
 ```
1. Replace this block with your solution.
1. Test your solution by running the cells following your block (indicated by ##TEST##)
1. Click the "Validate" button above to validate the work.

**Notes**
* You may add other cells and functions as needed
* Keep all code in the same notebook
* In order to receive credit, code must "Validate" on the JupyterHub server

---

In [1]:
class ZScoreTransformer
  attr_reader :means, :stdevs
  
  def initialize feature_names
    @means = Hash.new
    @stdevs = Hash.new
    @feature_names = feature_names    
  end
  
  def train dataset
       
    examples = dataset["data"] 

    j = 0
    data = Array.new(@feature_names.length) { Array.new(examples.length) }
    while j < @feature_names.length do
          i = 0
          while i < examples.length do
            data[j][i]=(examples[i]["features"][@feature_names[j]])

            i +=1
          end
          
          m = mean data[j].reject { |e| e.nil? or e.is_a? String}
          s = stdev data[j].reject { |e| e.nil? or e.is_a? String}

          @means[@feature_names[j]] = m
          @stdevs[@feature_names[j]] = s

          j +=1
    end
    return @means,@stdevs
    # END YOUR CODE
  end
  
    def apply example_batch
    # BEGIN YOUR CODE

    i = 0
    feature =  (self.means).keys


    while i< feature.length do    

        if (self.means[feature[i]]).is_a? Numeric and   (self.stdevs[feature[i]]).is_a?Numeric and    (self.stdevs[feature[i]]) !=0
          j = 0
          while j<example_batch.length do
              if example_batch[j]["features"][feature[i]].is_a?Numeric
                example_batch[j]["features"][feature[i]] = (example_batch[j]["features"][feature[i]] - self.means[feature[i]])/self.stdevs[feature[i]]
              end
                j+=1
          end
        end
        i +=1
    end


    #END YOUR CODE
    return example_batch
    end
end

:apply

In [2]:
class MeanImputation
  attr_reader :means
  
  def initialize feature_names
    @means = Hash.new
    @feature_names = feature_names
  end
  
  def train dataset    
    # BEGIN YOUR CODE
    examples = dataset["data"] 

    j = 0
    data = Array.new(@feature_names.length) { Array.new(examples.length) }
    while j < @feature_names.length do
          i = 0
          while i < examples.length do
            data[j][i]=(examples[i]["features"][@feature_names[j]])

            i +=1
          end
          
          m = mean data[j].reject { |e| e.nil? or e.is_a? String}


          @means[@feature_names[j]] = m


          j +=1
          
    
    end

    return @means
    #END YOUR CODE
  end
  
  def apply(example_batch)
    # BEGIN YOUR CODE
    i = 0
    example = Hash.new()
    
    while i< example_batch.length do    
        
        j = 0
        name  = example_batch[i]["features"].keys
        while j < @feature_names.length do   

          if name.include?@feature_names[j]
             if example_batch[i]["features"][@feature_names[j]].is_a?Numeric
               example_batch[i]["features"][@feature_names[j]] = example_batch[i]["features"][@feature_names[j]]
             else
               example_batch[i]["features"][@feature_names[j]] = self.means[@feature_names[j]]
             end
          else 
             example_batch[i]["features"].store(@feature_names[j],self.means[@feature_names[j]])

          end
        j+=1
        end

        i +=1
    end
    #END YOUR CODE
    return example_batch
  end
end

:apply

In [3]:
class AgeRangeAsVector
  def initialize; end
  def train dataset; end
  def apply(example_batch)
    min_age = 0
    max_age = 100
    feature_name = "days_birth"
    pattern = "age_range_%d"
    # BEGIN YOUR CODE
    i = 0
     
    while i< example_batch.length do        
          if example_batch[i]["features"].keys.include?feature_name
                bin = -example_batch[i]["features"][feature_name]/(365*5)  *5

                example_batch[i]["features"].delete(feature_name)

                binned_age = min_age

                while binned_age < max_age+1 do

                      new_feature_name = pattern% binned_age

                      if bin > binned_age or bin < (binned_age-5)    
                        if binned_age == max_age
                          example_batch[i]["features"].store(new_feature_name,1)
                        elsif binned_age == max_age
                          example_batch[i]["features"].store(new_feature_name,1)
                        else
                        example_batch[i]["features"].store(new_feature_name,nil)
                        end
                      else 
                        example_batch[i]["features"].store(new_feature_name,1)
                      end

                      binned_age+=5
                end
          else
          end
    i+=1
    end 

    
    #END YOUR CODE
    return example_batch
  end
end

:apply

In [4]:
class TargetAveraging
  attr_reader :means
  
  def initialize feature_names
    @means = Hash.new {|h,k| h[k] = Hash.new}
    @feature_names = feature_names
    @pattern = "avg_%s"
  end
  
  def train dataset    
    # BEGIN YOUR CODE
    i = 0
    while i< @feature_names.length do

          a = dataset.clone()["data"].select{|e|  !e["features"][@feature_names[i]].nil?}
          if a.empty?
          else
                b = a.uniq! {|e|  e["features"][@feature_names[i]] }
                name = b.map { |e|  e["features"][@feature_names[i]]  }           ######find unique value

                num =Array.new(name.length,0)
                lab_num = Array.new(name.length,0)
                j = 0
                while j< name.length do
                      a = dataset.clone()["data"].select{|e|  !e["features"][@feature_names[i]].nil?}
                      k = 0
                      while k< dataset["data"].length do
                            if dataset["data"][k]["features"][@feature_names[i]] == name[j]
                                  num[j]+=1
                                  lab_num[j] += dataset["data"][k]["label"]
                            end
                            k+=1
                      end

                      @means[@feature_names[i]].store(name[j],lab_num[j].to_f/num[j])
                      j+=1

                end
          end
          i+=1
    end
  
    return @means
    #END YOUR CODE
  end
  
    def apply(example_batch)
    # BEGIN YOUR CODE
    @pattern = "avg_%s"
    j = 0
    mean = self.means
    fea = mean.keys

    i = 0
    while i< fea.length do
      name = mean[fea[i]].keys
      j= 0
      
      while j<name.length do
        
        k = 0        
        while k < example_batch.length do

          if !(example_batch[k]["features"][fea[i]]).nil?
            value = example_batch[k]["features"][fea[i]]
            example_batch[k]["features"].delete(fea[i])

            example_batch[k]["features"].store(@pattern%fea[i],mean[fea[i]][value])
          end
          k+=1
        end
        
        j+=1
      end
      i+=1
    end

    
    
    #END YOUR CODE
    return example_batch
  end
  
end

:apply

In [5]:
class OneHotEncoding
  def initialize feature_names
    @feature_names = feature_names
    @pattern = "%s=%s"
  end
  
  def train dataset; end
  
  def apply(example_batch)
    
    # BEGIN YOUR CODE
    
    j = 0
    while j<example_batch.length do
      
          name = example_batch[j]["features"].keys
          i = 0
          while i< @feature_names.length do
              if name.include?@feature_names[i] and !(example_batch[j]["features"][@feature_names[i]]).nil?

                name_new = @pattern % [@feature_names[i], example_batch[j]["features"][@feature_names[i]]]
                example_batch[j]["features"].delete(@feature_names[i])
                example_batch[j]["features"].store(name_new,1.0)
              end
              i+=1
          end
          j+=1
    end
    
    #END YOUR CODE
    
    return example_batch
  end
end

:apply

In [6]:
class LogTransform
  def initialize feature_names
    @feature_names = feature_names
    @pattern = "log_%s"
  end
  
  def train dataset; end
  
  def apply(example_batch)
    # BEGIN YOUR CODE
    i = 0
    while i < @feature_names.length do
      j = 0
     
      while j<example_batch.length do
   
        if (example_batch[j]["features"].keys).include? @feature_names[i] and  example_batch[j]["features"][@feature_names[i]] > 0
                data = example_batch[j]["features"][@feature_names[i]]
                example_batch[j]["features"].delete(@feature_names[i])
                example_batch[j]["features"].store(@pattern % @feature_names[i], Math.log(data))
        end
        j+=1
      end
      
      
      i+=1
    end
    
    #END YOUR CODE
    return example_batch
  end
end

:apply

In [7]:
class L2Normalize
  def train dataset; end
  def apply(example_batch)
    # BEGIN YOUR CODE
      j = 0
      while j < example_batch.length do

          name = example_batch[j]["features"].keys
          i = 0
          normal_factor = norm example_batch[j]["features"]

          while i<name.length do
            if example_batch[j]["features"][name[i]].is_a? Numeric and normal_factor!=0
                example_batch[j]["features"][name[i]] = example_batch[j]["features"][name[i]]/normal_factor
            else
                example_batch[j]["features"][name[i]] = example_batch[j]["features"][name[i]]
            end
            i+=1
          end


          j+=1
      end    
    
    
    #END YOUR CODE
    return example_batch
  end
end

:apply

In [8]:
class FeatureTransformPipeline
  def initialize *transformers
    @transformers = transformers
  end
  
  def train dataset
    # BEGIN YOUR CODE 
    
    i = 0

    while i< @transformers.length do
        @transformers[i].train dataset 

        j = 0
        while j<dataset["data"].length do
  
          c = dataset["data"][j]
          dataset["data"][i] = (@transformers[i].apply  [c])[0]

          j+=1
        end
      i+=1
    end
       
    return dataset
    
    #END YOUR CODE
  end
  
  def apply example_batch 
    return @transformers.inject(example_batch) do |u, transform|
      u = transform.apply example_batch

    end
  end
end

:apply

# Final Project Tests

In [Part 3](./part-3.ipynb) of the final project, you will create your own features and try different combinations of models. This notebook sets up the foundation for the final project. Most of the code you need for the final project has already been implemented by you across multiple assignments. It has been scattered and we have copied and pasted where necessary. 

In this assignment, you will assemble a small library of the ```Learner```s, ```FeatureTransformer```s and ```Metric```s needed for the final project. The tests here do not cover everything we did in the course or everything you need for the final project, but it is a good start. Use this notebook to test your implementations. Once you are done, you can copy the appropriate code into the part-3 notebooks.

### Validation
If your Validate button shows an Interrupt error, that means your code takes longer than 1 minute to execute. This can happen. If you really want to do validation, use the command line following these steps. 

1. Go to the folder view and open a terminal.
1. Start a ```bash``` shell.
1. Run ```nbgrader validate final-project-3.1/tests-3.1.ipynb``` or any other path to a notebook.

In [9]:
require './final_project_lib.rb'
require './metrics.rb'
def load_german_credit_dataset; JSON.parse(File.read('german-credit.json')); end
def load_spambase_dataset; JSON.parse(File.read('spambase.json')); end

"if(window['d3'] === undefined ||\n   window['Nyaplot'] === undefined){\n    var path = {\"d3\":\"https://cdnjs.cloudflare.com/ajax/libs/d3/3.5.5/d3.min\",\"downloadable\":\"http://cdn.rawgit.com/domitry/d3-downloadable/master/d3-downloadable\"};\n\n\n\n    var shim = {\"d3\":{\"exports\":\"d3\"},\"downloadable\":{\"exports\":\"downloadable\"}};\n\n    require.config({paths: path, shim:shim});\n\n\nrequire(['d3'], function(d3){window['d3']=d3;console.log('finished loading d3');require(['downloadable'], function(downloadable){window['downloadable']=downloadable;console.log('finished loading downloadable');\n\n\tvar script = d3.select(\"head\")\n\t    .append(\"script\")\n\t    .attr(\"src\", \"http://cdn.rawgit.com/domitry/Nyaplotjs/master/release/nyaplot.js\")\n\t    .attr(\"async\", true);\n\n\tscript[0][0].onload = script[0][0].onreadystatechange = function(){\n\n\n\t    var event = document.createEvent(\"HTMLEvents\");\n\t    event.initEvent(\"load_nyaplot\",false,false);\n\t    win

:load_spambase_dataset

## Metrics

Copy / refactor **your** implementations of the following methods in the cell below located in this directory. Some are already in the file ```metrics.rb``` for you. Note that you should copy the entire method / class to the cell below.

* class AUCMetric 
* cross_validate
* mean
* stdev


In [10]:
def cross_validate dataset, folds, &block
  examples = dataset["data"]
  # BEGIN YOUR CODE
  batch = examples.length/folds
  folds.times do |fold|


      test_data = dataset.clone
      test_data["data"] = test_data["data"][fold*batch,batch]  ##CV training examples

      train_data = dataset.clone
      train_data["data"] = train_data["data"] - test_data["data"]  ##CV testing examples

    ## Call the callback like this:
    yield train_data, test_data, fold
  end
  
  
  #END YOUR CODE
end

:cross_validate

In [11]:
def score_binary_classification_model(data, weights, model)
  # BEGIN YOUR CODE

  scores =Array.new(data.length) { Array.new(2)}
  i = 0
  while i< data.length do
    scores[i][0] = model.predict(data[i],weights)
    scores[i][1] = data[i]["label"]
    i+=1
  end
  
  
  
  #END YOUR CODE
  return scores
end

:score_binary_classification_model

In [12]:
# BEGIN YOUR CODE

class TransformingLearner
  include Learner
  attr_accessor :name
  def initialize transformer, learner
    @parameters = learner.parameters
    @parameters["learner"] = learner.parameters["name"] || learner.class.name
    @transformer = transformer
    @learner = learner
    @name = self.class.name
  end
  def train dataset
    @transformer.train dataset
    transformed_examples = @transformer.apply dataset["data"]
    train_dataset = dataset.clone
    train_dataset["data"] = transformed_examples
    @learner.train train_dataset
  end
  
  def predict example
    transformed_example = @transformer.apply [example]
    @learner.predict transformed_example.first
  end
  
  def evaluate dataset
    transformed_dataset = dataset.clone
    transformed_dataset["data"] = @transformer.apply dataset["data"]
    @learner.evaluate transformed_dataset
  end
end

class CopyingTransformingLearner
  include Learner
  attr_accessor :name
  def initialize transformer, learner
    @parameters = learner.parameters
    @parameters["learner"] = learner.parameters["name"] || learner.class.name
    @transformer = transformer
    @learner = learner
    @name = self.class.name
  end

  def clone_example example
    e = example.clone
    e["features"] = example["features"].clone
    return e
  end
    
  def clone_dataset dataset
    cloned_dataset = dataset.clone
    cloned_dataset["features"] = dataset["features"].clone
    cloned_dataset["data"] = dataset["data"].map {|e| clone_example(e)}
    return cloned_dataset
  end
    
  def train dataset
    @transformer.train clone_dataset(dataset)
      
    train_dataset = clone_dataset(dataset)
    transformed_examples = @transformer.apply train_dataset["data"]
    train_dataset["data"] = transformed_examples
    @learner.train train_dataset
  end
  
  def predict example
    transformed_example = @transformer.apply [clone_example(example)]
    @learner.predict transformed_example.first
  end
  
  def evaluate dataset
    transformed_dataset = clone_dataset dataset
    transformed_dataset["data"] = @transformer.apply dataset["data"]
    @learner.evaluate transformed_dataset
  end
end



#END YOUR CODe

:evaluate

In [13]:
def mean x
  # BEGIN YOUR CODE
  x = x.compact
  x.inject(0.0){|sum,i| sum + i }/x.length()  
  #END YOUR CODE
end

:mean

In [14]:
def stdev x
  # BEGIN YOUR CODE
  x = x.compact
  mu = mean x
  su = x.inject(0.0){|sum,i| sum + (i-mu)**2 }/(x.length()-1)
  Math.sqrt(su) 
  #END YOUR CODE
end

:stdev

In [15]:
#Implement the error function given a weight vector, w
def dot x, w
  # BEGIN YOUR CODE
    x_name = x.keys
    w_name = w.keys

    i = 0
    dot = 0.0
    if x_name.length == 0 
      dot =  0.0
    else 
        while i<x_name.length do

          if w_name.include?x_name[i]
            index = w_name.index { |x| [x_name[i]].include?(x) }
            if x[x_name[i]].is_a? Numeric  and w[w_name[index]].is_a? Numeric
              dot += x[x_name[i]]*w[w_name[index]]
            else 
              dot +=0.0
            end
          else 
            dot += 0.0
          end
          i+=1
        end  
    end
  return dot
  #END YOUR CODE
end


:dot

In [16]:
def norm w
  # BEGIN YOUR CODE
  return Math.sqrt(dot w, w)
  #END YOUR CODE
end

:norm

In [17]:
def confusion_matrix(scores, t)
  matrix = Hash.new {|h,predicted_class| h[predicted_class] = Hash.new {|h,true_class| h[true_class] = 0.0}}
  # BEGIN YOUR CODE
  matrix["N"]["N"] = 0
  matrix["N"]["P"] = 0
  matrix["P"]["N"] = 0
  matrix["P"]["P"] = 0
  i = 0
  while i<scores.length do
    if scores[i][1]>0 and scores[i][0]>=t
      matrix["P"]["P"] += 1
    elsif scores[i][1]>0 and scores[i][0]<t
      matrix["N"]["P"] += 1
    elsif scores[i][1]<=0 and scores[i][0]>=t
      matrix["P"]["N"] += 1
    elsif scores[i][1]<=0 and scores[i][0]<t
      matrix["N"]["N"] += 1
    end
    i+=1
  end
  
  #END YOUR CODE
  return matrix
end


:confusion_matrix

In [18]:
class ZScoreTransformer
  attr_reader :means, :stdevs
  
  def initialize feature_names
    @means = Hash.new
    @stdevs = Hash.new
    @feature_names = feature_names    
  end
  
  def train dataset
       
    examples = dataset["data"] 

    j = 0
    data = Array.new(@feature_names.length) { Array.new(examples.length) }
    while j < @feature_names.length do
          i = 0
          while i < examples.length do
            data[j][i]=(examples[i]["features"][@feature_names[j]])

            i +=1
          end
          
          m = mean data[j].reject { |e| e.nil? or e.is_a? String}
          s = stdev data[j].reject { |e| e.nil? or e.is_a? String}

          @means[@feature_names[j]] = m
          @stdevs[@feature_names[j]] = s

          j +=1
    end
    return @means,@stdevs
    # END YOUR CODE
  end
  
    def apply example_batch
    # BEGIN YOUR CODE

    i = 0
    feature =  (self.means).keys

 
    while i< feature.length do    

        if (self.means[feature[i]]).is_a? Numeric and   (self.stdevs[feature[i]]).is_a?Numeric and    (self.stdevs[feature[i]]) !=0
          j = 0
          while j<example_batch.length do
              if example_batch[j]["features"][feature[i]].is_a?Numeric
                example_batch[j]["features"][feature[i]] = (example_batch[j]["features"][feature[i]] - self.means[feature[i]])/self.stdevs[feature[i]]
              end
                j+=1
          end
        end
        i +=1
    end
          

    #END YOUR CODE
    return example_batch
  end
end

:apply

## Linear Models
Convert your regularized logistic regression implementation into a ```LogisticRegressionLearner```, stored in ```"linear_models.rb"```. You should reuse the ```StochasticGradientDescent``` and ```LogisticRegressionModelL2``` classes from previous assignments. 

The ```Learner``` interface (implemented as a ruby mixin) exposes common methods that all classifiers will use. The ```Learner``` is defined as follows:
```
module Learner  
  attr_reader :parameters
  def name
      self.class.name
  end
  def train train_dataset    
  end
  def predict example
  end
  def evaluate eval_dataset
  end
end
```
where ```parameters``` contain, say the weights or other hyperparameters.

Implement the ```LogisticRegressionLearner``` to like this:

```ruby
class LogisticRegressionLearner
  attr_reader :parameters
  attr_reader :weights  
  include Learner  
  
  def initialize regularization: 0.0, learning_rate: 0.01, batch_size: 20, epochs: 1
    @parameters = {"regularization" => regularization, 
      "learning_rate" => learning_rate, 
      "epochs" => epochs, "batch_size" => batch_size}
  end
    
  def train dataset
      ###
  end
    
  def predict example
      ###
  end
    
  def evaluate dataset
      ###
  end
```



In [19]:
# BEGIN YOUR CODE
class StochasticGradientDescent
  attr_reader :weights
  attr_reader :objective
  def initialize obj, w_0, lr 
    @objective = obj
    @weights = w_0
    @n = 1.0
    @lr = lr
  end
  def update x
    # BEGIN YOUR CODE

    lr  = @lr.to_f / Math.sqrt(@n)
    @n +=1

    if x.empty?
    else
      feature = x[0]["features"].keys
    
    end
  
    if (@weights.keys)==[0] and  x.length>0
      i=0
      while i<feature.length do
        @weights.store(feature[i],@weights[0])
        i+=1
      end
      
    end
    
    @weights.delete(0)
    
    i = 0
    gradient = @objective.grad x,@weights

    while i < @weights.length do
      if @weights[@weights.keys[i]].is_a?Numeric and gradient[@weights.keys[i]].is_a?Numeric
        @weights[@weights.keys[i]]  = @weights[@weights.keys[i]] - lr*gradient[@weights.keys[i]]        
      end
      i+=1
    end    

   return @objective.adjust @weights

  
    #END YOUR CODE
  end
end
#END YOUR CODE

:update

In [20]:
class LogisticRegressionModelL2
  def initialize reg_param
    @reg_param = reg_param
  end

  def predict row, w
    x = row["features"]    
    1.0 / (1 + Math.exp(-dot(w, x)))
  end
  
  def adjust w
    w.each_key {|k| w[k] = 0.0 if w[k].nan? or w[k].infinite?}
    w.each_key {|k| w[k] = 0.0 if w[k].abs > 1e5 }
  end
  
  def func data, w
    # BEGIN YOUR CODE
    adjust w
    i = 0
    sum  = 0.0

    while i< data.length do
      y=(data[i]["label"])
      yhat=dot(w, data[i]["features"])
      sum += Math.log(Math.exp(-y*yhat)+1.0)
      i+=1
    end

    
    return sum/data.length + 0.5 * @reg_param *(norm(w)**2)
  end 
   def grad data, w
    # BEGIN YOUR CODE
    
    g = Hash.new()
    key_list =[]
    data.each do |e|
        key_list.push((e["features"].keys).compact)
    end
    name = key_list.flatten.compact.uniq{|x| x  }

    sum  = Array.new(name.length)
    

    adjust w
    
    if (w.keys).empty?
      i = 0
      while i<name.length do
        w.store(name[i],w[0])
        i+=1
      end
      w.delete(0)
    end
    
    norm  = norm w

    i = 0
    while i< w.length do
      
      sum = 0.0
      j = 0
      while j<data.length
        yhat =dot  data[j]["features"], w
        ylabel = data[j]["label"]
        if data[j]["features"][name[i]].is_a? Numeric
          sum += (1.0/(1.0+Math.exp(ylabel*yhat)))*(-ylabel *data[j]["features"][name[i]] )
        end
        j+=1
      end
      sum = (sum/data.length) + @reg_param * w[name[i]]
      g.store(name[i],sum)
      i+=1
    end
    
    
    #END YOUR CODE
    return g
  end
  def adjust w
    w.each_key {|k| w[k] = 0.0 if w[k].nan? or w[k].infinite?}
    w.each_key {|k| w[k] = 0.0 if w[k].abs > 1e5 }
  end
end

:adjust

In [21]:
# BEGIN YOUR CODE
class LogisticRegressionLearner
  attr_reader :parameters
  attr_reader :weights  
  include Learner  

  def initialize regularization: 0.0, learning_rate: 0.01, batch_size: 20, epochs: 1
    @parameters = {"regularization" => regularization, 
      "learning_rate" => learning_rate, 
      "epochs" => epochs, "batch_size" => batch_size}
  end

  def train dataset


    reg = @parameters["regularization"]
    batch_size = @parameters["batch_size"]
    num_epoch = @parameters["epochs"]
    lr = @parameters["learning_rate"]

    w = @weights


    if w.nil?
      w = Hash.new {|h,k| h[k] =0.0}
    end

   losses = []

   num_epoch.times do

        obj = LogisticRegressionModelL2.new(reg)
        l = obj.func(dataset["data"][0,batch_size],w)
  
        losses.append(l)
        sgd = StochasticGradientDescent.new(obj, w, lr)
        sgd.update(dataset["data"][0,batch_size])

        w = sgd.weights

  
  end
  @weights = w
  
  return w
   
  end


  def predict example 
    
    x = example["features"]    
   
   return dot(@weights, x)
  end


  def evaluate dataset
   
    examples = dataset["data"]
    
    i = 0
    score = Array.new(examples.length) { Array.new(2) }
    while i<examples.length do      
      x = examples[i]   
      y = self.predict(x)   
      score[i][0] = y
      score[i][1] = x["label"]
      i+=1
    end

  return score
  end


end
#END YOUR CODE


:evaluate

In [22]:
def num_positives scores
  # BEGIN YOUR CODE

  i = 0
  sum = 0
  while i< scores.length do
    if scores[i][1]>0
      sum += 1
    end
    i+=1
  end
  return sum
  
  #END YOUR CODE
end

:num_positives

In [23]:
def num_negatives scores
  # BEGIN YOUR CODE

  return scores.length - num_positives(scores)
  
  #END YOUR CODE
end

:num_negatives

In [24]:

class AUCMetric 
  include Metric  

  def roc_curve(scores)
  fp_rates = [0.0]
  tp_rates = [0.0]
  auc = 0.0

  # BEGIN YOUR CODE


  t_list = scores.sort_by{|e| [-e[0]]}
 
  i = 0
  total_pos = num_positives scores
  total_neg = num_negatives scores
  p = 0.0
  n = 0.0

  t_list.each do |e|

    if e[1] == 1
      p +=1.0   
      fp_rates.append(fp_rates[-1])      
      tp_rates.append(p/total_pos)
    else
      n +=1.0 
      fp_rates.append(n/total_neg)      
      tp_rates.append(tp_rates[-1])
    end

  end

  i = 1
  while i < (tp_rates.length) do
      auc+= 0.5*(fp_rates[i] -fp_rates[i-1])*(tp_rates[i] + tp_rates[i-1]  )
    i+=1
  end
  

  #END YOUR CODE
    return [fp_rates, tp_rates, auc]
  end



#   def calc_auc_only scores
#     fp, tp, auc = roc_curve scores
#     return auc
#   end
  def apply scores
    fp, tp, auc = roc_curve scores
    return auc
  end

  #END YOUR CODE
end

:apply

## Transformers
Refactor / copy your ```FeatureTransformer``` implementations to the cell below. Extra transformers has been implemented for you in a file ```transformers.rb``` to demonstrate how to apply feature transforms before training learners. There is also a wrapped learner that can transform data and then train any learner. 

In [25]:
# BEGIN YOUR CODE
def feature_transformer
  FeatureTransformPipeline.new(
    #ext_source
    ZScoreTransformer.new(%w(ext_source_1 ext_source_2 ext_source_3)),
    MeanImputation.new(%w(ext_source_1 ext_source_2 ext_source_3)),
    
    #Treat amt_income_total and amt_credit as log normal
    LogTransform.new(%w(amt_income_total amt_credit)),
    ZScoreTransformer.new(%w(log_amt_income_total log_amt_credit)),
      
    #Imputation for commonarea_avg
    MeanImputation.new(%w(commonarea_avg)),      #### log_amt_income_total log_amt_credit ?
      
    #One-hot encoded features
    AgeRangeAsVector.new,      
    OneHotEncoding.new(%w(name_family_status code_gender)),
      
    #Target Averages
    TargetAveraging.new(%w(name_income_type flag_own_car flag_own_realty
      name_family_status organization_type name_housing_type name_education_type)),      
    L2Normalize.new
  )
end


#END YOUR CODE

:feature_transformer

In [26]:
def FeatureTransformPipeline
  def initialize *transformers
    @transformers = transformers
  end
  
  def train dataset
    # BEGIN YOUR CODE 
    
    i = 0

    while i< @transformers.length do
        @transformers[i].train dataset 

        j = 0
        while j<dataset["data"].length do
  
          c = dataset["data"][j]
          dataset["data"][i] = (@transformers[i].apply  [c])[0]

          j+=1
        end
      i+=1
    end
       
    return dataset
    
    #END YOUR CODE
  end
  
  def apply example_batch 
    return @transformers.inject(example_batch) do |u, transform|
      u = transform.apply example_batch

    end
  end
end

:FeatureTransformPipeline

In [27]:
#  spambase = load_spambase_dataset()  
#   linear = LogisticRegressionLearner.new(regularization: 0.01, learning_rate: 0.001, batch_size: 20, epochs: 1)
  
#   ## Provide your own name to a TransformingLearner
#   zlearner = CopyingTransformingLearner.new(ZScoreTransformer.new(spambase["features"]), linear)  
#   zlearner.name = "ZScore_Logistic"
  
#   learners = [zlearner]
#   puts "debug",learners,"debug"
#   df, learner_summary = parameter_search learners, spambase
#   best_model_stats = learner_summary["ZScore_Logistic"]

In [28]:
def test_0f2399()
  spambase = load_spambase_dataset()  
  linear = LogisticRegressionLearner.new(regularization: 0.01, learning_rate: 0.001, batch_size: 20, epochs: 1)
  
  ## Provide your own name to a TransformingLearner
  zlearner = CopyingTransformingLearner.new(ZScoreTransformer.new(spambase["features"]), linear)  
  zlearner.name = "ZScore_Logistic"
  
  learners = [zlearner]
  
  df, learner_summary = parameter_search learners, spambase
  best_model_stats = learner_summary["ZScore_Logistic"]
  assert_true best_model_stats["mean_test_metric"] > 0.6, "Must return AUC > 0.6"
  
  df
end
test_0f2399()

5-fold CV: CopyingTransformingLearner, parameters: {"regularization"=>0.01, "learning_rate"=>0.001, "epochs"=>1, "batch_size"=>20, "learner"=>"LogisticRegressionLearner"}
0.624034272645382
{
  "ZScore_Logistic": {
    "learner": "ZScore_Logistic",
    "parameters": {
      "regularization": 0.01,
      "learning_rate": 0.001,
      "epochs": 1,
      "batch_size": 20,
      "learner": "LogisticRegressionLearner"
    },
    "folds": 5,
    "mean_train_metric": 0.621668199631282,
    "stdev_train_metric": 0.024901567194550795,
    "mean_test_metric": 0.624034272645382,
    "stdev_test_metric": 0.027451654221655422
  }
}


,learner,parameters,folds,mean_train_metric,stdev_train_metric,mean_test_metric,stdev_test_metric
0,ZScore_Logistic,"{""regularization""=>0.01, ""learning_rate""=>0.001, ""epochs""=>1, ""batch_size""=>20, ""learner""=>""LogisticRegressionLearner""}",5,0.621668199631282,0.024901567194550795,0.624034272645382,0.027451654221655422


In [29]:
def test_ef8eed()
  german_credit = load_german_credit_dataset()  
  
  linear = LogisticRegressionLearner.new(regularization: 0.0001, learning_rate: 0.01, batch_size: 20, epochs: 50)    
  transformer = FeatureTransformPipeline.new(
    OneHotEncoding.new(%w(checking_account credit_history purpose savings job_tenure)),
    OneHotEncoding.new(%w(personal_status_gender other_debtors property other_installments housing)),
    OneHotEncoding.new(%w(job has_telephone is_foreign_worker)),
    ZScoreTransformer.new(%w(loan_duration installment_to_salary residence_tenure age)),
    LogTransform.new(%w(credit_amount))
  )
  
  zlearner = CopyingTransformingLearner.new(transformer, linear)  
  zlearner.name = "OneHot_ZScore_Log_Logistic"
  learners = [zlearner]
  df, best_model = parameter_search learners, german_credit

  best_model_stats = best_model[zlearner.name]
  assert_true best_model_stats["mean_test_metric"] > 0.65, "Must return AUC > 0.65"
end
test_ef8eed()

5-fold CV: CopyingTransformingLearner, parameters: {"regularization"=>0.0001, "learning_rate"=>0.01, "epochs"=>50, "batch_size"=>20, "learner"=>"LogisticRegressionLearner"}
0.6719494060200436
{
  "OneHot_ZScore_Log_Logistic": {
    "learner": "OneHot_ZScore_Log_Logistic",
    "parameters": {
      "regularization": 0.0001,
      "learning_rate": 0.01,
      "epochs": 50,
      "batch_size": 20,
      "learner": "LogisticRegressionLearner"
    },
    "folds": 5,
    "mean_train_metric": 0.6875603552194638,
    "stdev_train_metric": 0.08545420079756087,
    "mean_test_metric": 0.6719494060200436,
    "stdev_test_metric": 0.1068412459299978
  }
}


## Putting it together

Now, we will test different type models on the same dataset and compare. This is similar to what will happen in the final project, so use this test as an example.

In [30]:
def test_a0551a()
  german_credit = load_german_credit_dataset()

  linear1 = LogisticRegressionLearner.new(regularization: 0.0001, learning_rate: 0.01, batch_size: 20, epochs: 50)    
  transformer = FeatureTransformPipeline.new(
    OneHotEncoding.new(%w(checking_account credit_history purpose savings job_tenure)),
    OneHotEncoding.new(%w(personal_status_gender other_debtors property other_installments housing)),
    OneHotEncoding.new(%w(job has_telephone is_foreign_worker)),
    ZScoreTransformer.new(%w(loan_duration installment_to_salary residence_tenure age)),
    LogTransform.new(%w(credit_amount))
  )
  learner1 = CopyingTransformingLearner.new(transformer, linear1)
  learner1.name = "logistic-regression-0.0001"
  
  linear2 = LogisticRegressionLearner.new(regularization: 0.1, learning_rate: 0.01, batch_size: 20, epochs: 50)    
  learner2 = CopyingTransformingLearner.new(transformer, linear2)
  learner2.name = "logistic-regression-0.1"
  
  learners = [learner1, learner2] 
  df, model_stats = parameter_search learners, german_credit
  
  assert_true model_stats["logistic-regression-0.0001"]["mean_test_metric"] > 0.65, "Logistic Regression 1 > 0.65"

  auc1 = model_stats["logistic-regression-0.0001"]["mean_test_metric"]
  auc2 = model_stats["logistic-regression-0.1"]["mean_test_metric"]
  
  assert_true auc1 > auc2, "Linear Model with transforms beats Decision Tree out of the box"  
  df
end
test_a0551a()

5-fold CV: CopyingTransformingLearner, parameters: {"regularization"=>0.0001, "learning_rate"=>0.01, "epochs"=>50, "batch_size"=>20, "learner"=>"LogisticRegressionLearner"}
0.6719494060200436
5-fold CV: CopyingTransformingLearner, parameters: {"regularization"=>0.1, "learning_rate"=>0.01, "epochs"=>50, "batch_size"=>20, "learner"=>"LogisticRegressionLearner"}
0.6715707131371996
{
  "logistic-regression-0.0001": {
    "learner": "logistic-regression-0.0001",
    "parameters": {
      "regularization": 0.0001,
      "learning_rate": 0.01,
      "epochs": 50,
      "batch_size": 20,
      "learner": "LogisticRegressionLearner"
    },
    "folds": 5,
    "mean_train_metric": 0.6875603552194638,
    "stdev_train_metric": 0.08545420079756087,
    "mean_test_metric": 0.6719494060200436,
    "stdev_test_metric": 0.1068412459299978
  },
  "logistic-regression-0.1": {
    "learner": "logistic-regression-0.1",
    "parameters": {
      "regularization": 0.1,
      "learning_rate": 0.01,
      "ep

,learner,parameters,folds,mean_train_metric,stdev_train_metric,mean_test_metric,stdev_test_metric
0,logistic-regression-0.0001,"{""regularization""=>0.0001, ""learning_rate""=>0.01, ""epochs""=>50, ""batch_size""=>20, ""learner""=>""LogisticRegressionLearner""}",5,0.6875603552194638,0.08545420079756087,0.6719494060200436,0.1068412459299978
1,logistic-regression-0.1,"{""regularization""=>0.1, ""learning_rate""=>0.01, ""epochs""=>50, ""batch_size""=>20, ""learner""=>""LogisticRegressionLearner""}",5,0.6863714492550004,0.08512325619852246,0.6715707131371996,0.10715379554132377
